In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
import torch
torch.manual_seed(0)

import random
random.seed(0) 

import numpy as np
np.random.seed(0)

In [3]:
# Set patient and record (same channels)

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
observations = ["chb01_03", "chb01_04"]#, "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]

# observations = ["chb02_16", "chb02_19"]

# observations = ["chb04_28"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN14_1", "PN14_2", "PN14_3", "PN14_4"]

# observations = ["PN14_4"]
# observations = ["chb01_16"]

create_dataset_json(observations)

In [4]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache data

In [5]:
remove_cache = True

if remove_cache:
    list_folders = [".\data\cache\datasets", ".\data\cache\explainers", ".\data\cache\oracles"]

    for folder in list_folders:
        for file in os.listdir(folder):
            if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
                cache_path = os.path.join(folder, file)
                if os.path.isfile(cache_path):
                    os.remove(cache_path)
                    print(f"Removed file: {cache_path}")
                elif os.path.isdir(cache_path):
                    shutil.rmtree(cache_path)
                    print(f"Removed folder: {cache_path}")

Removed file: .\data\cache\datasets\GCS-cc6bf7703feb7a109eb796a73c8a7693
Removed folder: .\data\cache\oracles\GCS-cc6bf7703feb7a109eb796a73c8a7693


## Run Gretel experiment

In [6]:
# NOTICE: If it crashes, check if there are any OracleTorch files from the future!

In [7]:
path = 'config/GCS-GCN-topk.jsonc'
# path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-perfect.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# # Additional stuff
# print('--------------------------------------')

# for i in range(len(eval_manager._evaluators)):
#     # context.logger.info(f"graph_ids: {eval_manager._evaluators[i].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
#     context.logger.info(f"graph_metrics: {eval_manager._evaluators[i].get_instance_and_counterfactual_graph_metrics()}")

['chb01_03', 'chb01_04']
Generating context for: config/GCS-GCN-topk.jsonc
2025-06-20 16:05:01,109 | INFO | 20736 - Executing: config/GCS-GCN-topk.jsonc Run: -1
2025-06-20 16:05:01,158 | INFO | 20736 - Creating the evaluation manager.......................................................
2025-06-20 16:05:01,192 | INFO | 20736 - Creating the PAIRED evaluators...............................................................
2025-06-20 16:05:01,294 | INFO | 20736 - Creating: GCS-cc6bf7703feb7a109eb796a73c8a7693
2025-06-20 16:05:01,294 | INFO | 20736 - Instantiating: src.dataset.generators.gcs.GCS
2025-06-20 16:05:04,183 | INFO | 20736 - Saved: GCS-cc6bf7703feb7a109eb796a73c8a7693
2025-06-20 16:05:04,234 | INFO | 20736 - Created: GCS-cc6bf7703feb7a109eb796a73c8a7693
2025-06-20 16:05:04,257 | INFO | 20736 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-06-20 16:05:04,270 | INFO | 20736 - Instantiating: torch.optim.RMSprop
2025-06-20 16:05:04,272 | INFO | 20736 - Instantiating: torch.nn.

In [8]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [9]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 20/06/2025, ore 16:10


In [10]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [11]:
# DA METTERE DI NUOVO

for i in range(len(eval_manager._evaluators)):
    cf_dict = {}

    for instance in eval_manager._evaluators[i].dataset.instances:
        if instance.label == 1:
            print(instance.id)
            lista = eval_manager._evaluators[i]._explainer.explain(instance, return_list=True)
            cf_dict[instance.id] = lista

    # Salva cf_dict in un file chiamato 'cf_dict.pkl'
    with open(f"cf_dict_{file_name}_{i}.pkl", 'wb') as f:
        pickle.dump(cf_dict, f)

341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590


In [12]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)